In [2]:
# Gerekli kütüphaneleri yükleyin
import pandas as pd
import re

In [11]:
# pandasın ayarlarını güncelleyerek her kolonu ve ilk 10 satırını vermesini sağlıyoruz
pd.set_option('display.max_columns', None)  # Tüm kolonları göster
pd.set_option('display.max_rows', 10)  # Her kolonun ilk 10 satırını göster

# Log dosyasını yükleyin
file_path = './web_traffic.log'
with open(file_path, 'r') as file:
    logs = file.readlines()

# Log verilerini DataFrame'e dönüştür
def parse_log(log_line):
    # Log formatına uygun düzenli ifadeyi kullanarak verileri ayır
    pattern = r'(\S+) - - \[(.*?)\] "(\S+) (\S+) HTTP/1.1" (\d+) (\d+)'
    match = re.match(pattern, log_line)
    if match:
        ip, timestamp, method, endpoint, status, size = match.groups()
        return ip, timestamp, method, endpoint, status, size
    return None

# Verileri ayıklama ve DataFrame oluşturma
data = [parse_log(line) for line in logs]
df = pd.DataFrame(data, columns=['IP', 'Timestamp', 'Method', 'Endpoint', 'Status', 'Size'])

# İlk birkaç satırı görüntüle
print(df.head(10))


                IP                   Timestamp  Method             Endpoint  \
0      32.32.61.41  05/Jun/2001:03:32:13 +0000    POST               /about   
1     31.156.84.68  31/Aug/1996:05:06:28 +0000    POST              /signup   
2  111.224.206.188  26/Dec/1974:05:31:04 +0000     PUT  /products/item?id=2   
3       3.35.216.7  08/Apr/2023:10:38:31 +0000    POST            /products   
4    43.163.88.139  02/Jul/2020:17:50:02 +0000  DELETE               /login   
5   14.177.206.174  16/Aug/1985:06:05:49 +0000     PUT  /products/item?id=1   
6   152.231.83.225  18/Sep/1979:16:15:04 +0000    POST              /signup   
7   151.40.216.114  08/May/1979:18:36:27 +0000    POST                /home   
8     27.9.236.143  05/Sep/2002:09:04:22 +0000  DELETE               /about   
9     22.156.71.77  06/Jun/1975:06:37:42 +0000    POST            /products   

  Status  Size  
0    301  1363  
1    200  2512  
2    500  1154  
3    200  2962  
4    500   207  
5    200  2180  
6    301   

# Web Trafik Logları Veri Açıklaması

Bu belgede, web trafik loglarının içerdiği kolonlar ve verilerin detayları açıklanacaktır.

## Veri Seti Kolonları

### 1. **IP**
- **Açıklama:** Web isteğini yapan istemcinin IP adresi.
- **Örnek:** `32.32.61.41`
- **Tür:** String

### 2. **Timestamp**
- **Açıklama:** Web isteğinin yapıldığı tarih ve saat.
- **Format:** `[dd/mmm/yyyy:hh:mm:ss +0000]`
- **Örnek:** `05/Jun/2001:03:32:13 +0000`
- **Tür:** String (Daha sonra tarih/zaman formatına dönüştürülecek)

### 3. **Method**
- **Açıklama:** Web isteğinde kullanılan HTTP yöntemi.
- **Değerler:** `GET`, `POST`, `PUT`, `DELETE`, vb.
- **Örnek:** `POST`
- **Tür:** String

### 4. **Endpoint**
- **Açıklama:** Web isteğinin yapıldığı URL'nin yolu.
- **Örnek:** `/about`, `/signup`, `/products/item?id=2`
- **Tür:** String

### 5. **Status**
- **Açıklama:** Web isteğine verilen HTTP yanıt durumu kodu.
- **Değerler:** `200` (Başarılı), `301` (Kalıcı Yönlendirme), `500` (Sunucu Hatası), vb.
- **Örnek:** `301`
- **Tür:** Integer

### 6. **Size**
- **Açıklama:** Web isteği yanıtının boyutu (bayt cinsinden).
- **Örnek:** `1363`, `2512`
- **Tür:** Integer


In [12]:
print(df.shape)
print(df.columns)
df.isnull().sum()

(10000, 6)
Index(['IP', 'Timestamp', 'Method', 'Endpoint', 'Status', 'Size'], dtype='object')


IP           0
Timestamp    0
Method       0
Endpoint     0
Status       0
Size         0
dtype: int64

In [13]:
df.nunique()

IP           10000
Timestamp    10000
Method           4
Endpoint         9
Status           4
Size          4209
dtype: int64